#### Task 1: Simple vector embedding generation

**Objective:**
Generate vector embeddings from text data.

**Task Description:**

- load huggingface embedding model (`model_name="sentence-transformers/all-mpnet-base-v2"`)
- embed simple text queries

How to select the right embedding model: [MTEB - Massive Text Embedding Benchmark](https://huggingface.co/blog/mteb)

**Useful links:**

- [Langchain Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)


In [1]:
from langchain_huggingface import HuggingFaceEmbeddings

# ADD HERE YOUR CODE
embedding_model = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")

/workspaces/TeamNovaBot/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/workspaces/TeamNovaBot/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
text = "This is a test document."

# ADD HERE YOUR CODE
# Perform vector search
query_vector = text

print(f"Embedding vector length: {len(query_vector)}")
print(query_vector[:10])

Embedding vector length: 24
This is a 


#### Task 2: Generate embedding vectors with custom dataset

**Objective:**
Load custom dataset, preprocess it and generate vector embeddings.

**Task Description:**

- load pdf document "AI_Book.pdf" via langchain document loader: `PyPDFLoader`
- use RecursiveCharacterTextSplitter to split documents into chunks
- generate embeddings for single documents

**RecursiveCharacterTextSplitter:**

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is `["\n\n", "\n", " ", ""]`. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

**Useful links:**

- [Langchain PyPDFLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html)
- [Langchain RecursiveCharacterTextSplitter](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)


In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
import re

pdf_doc = "./AI_Book.pdf"

# Create pdf data loader
# ADD HERE YOUR CODE
loader = PyPDFLoader(
    file_path = pdf_doc,
    # passwort = "my-passwort"
    extract_images = False,
)

# Load and split documents in chunks
# ADD HERE YOUR CODE
pages_chunked = RecursiveCharacterTextSplitter().split_documents(documents = loader.load())

# Function to clean text by removing invalid unicode characters, including surrogate pairs
def clean_text(text):
    # Remove surrogate pairs
    text = re.sub(r'[\ud800-\udfff]', '', text)
    # Optionally remove non-ASCII characters (depends on your use case)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    return text

def clean_and_create_document(chunk):
    cleaned_text = clean_text(chunk.page_content)
    return Document(page_content = cleaned_text, metadata = chunk.metadata)

pages_chunked_cleaned1 = [clean_and_create_document(chunk) for chunk in pages_chunked]
pages_chunked_cleaned = [clean_text(chunk.page_content) for chunk in pages_chunked]
print(pages_chunked[0])

page_content='Aurélien GéronHands-on Machine Learning with
Scikit-Learn, Keras, and
TensorFlow
Concepts, Tools, and Techniques to
Build Intelligent SystemsSECOND EDITION
Boston Farnham Sebastopol Tokyo Beijing Boston Farnham Sebastopol Tokyo Beijing' metadata={'source': './AI_Book.pdf', 'page': 2}


In [4]:
print(pages_chunked[1])

page_content='978-1-492-03264-9
[LSI]Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow
by Aurélien Géron
Copyright © 2019 Aurélien Géron. All rights reserved.
Printed in the United States of America.
Published by O’Reilly Media, Inc., 1005 Gravenstein Highway North, Sebastopol, CA 95472.
O’Reilly books may be purchased for educational, business, or sales promotional use. Online editions are
also available for most titles ( http://oreilly.com ). For more information, contact our corporate/institutional
sales department: 800-998-9938 or corporate@oreilly.com .
Editor:  Nicole Tache
Interior Designer:  David FutatoCover Designer:  Karen Montgomery
Illustrator:  Rebecca Demarest
June 2019:  Second Edition
Revision History for the Early Release
2018-11-05: First Release
2019-01-24: Second Release
2019-03-07: Third Release
2019-03-29: Fourth Release
2019-04-22: Fifth Release
See http://oreilly.com/catalog/errata.csp?isbn=9781492032649  for release details.
The O’Reilly logo 

In [5]:
print(f"Number of text chunks: {len(pages_chunked)}")

Number of text chunks: 507


#### Task 3: Store vector embeddings from pdf document to ChromaDB vector database.

**Objective:**

Store vector embeddings into ChromaDB to store knowledge.

**Task Description:**

- create chromadb client
- create chromadb collection
- create langchain chroma db client
- store text document chunks and vector embeddings to vector databases

**Useful links:**

- [Langchain How To](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#initialization-from-client)


In [6]:
from langchain_chroma import Chroma
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

client = chromadb.HttpClient(
    host     = "localhost",
    port     = 8000,
    ssl      = False,
    headers  = None,
    settings = Settings(allow_reset = True, anonymized_telemetry = False),
    tenant   = DEFAULT_TENANT,
    database = DEFAULT_DATABASE,
)

# ADD HERE YOUR CODE
# Create a collection
collection = client.get_or_create_collection("collection_name")

# ADD HERE YOUR CODE
# Create chromadb
vector_db_from_client = Chroma(
    client = client,
    collection_name = "collection_name",
    embedding_function = embedding_model,
)

In [7]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(pages_chunked[:50]))]
#uuids = [str(uuid4()) for _ in range(len(pages_chunked_cleaned1[:50]))]

# ADD HERE YOUR CODE
vector_db_from_client.add_documents(documents = pages_chunked[:50], id = uuids)
#vector_db_from_client.add_documents(documents = pages_chunked_cleaned1, id = uuids)

['2d933ecf-281a-487f-8b78-105a28416205',
 'de62461a-02dd-4214-991d-50d9581d0c78',
 '7a334369-c0a9-4570-8513-6f216983fb0e',
 'cf54b9ce-7375-4d3d-914b-f1bcf4707604',
 '8da8dcd0-f626-49fd-b4fe-ec47fa5b4a01',
 'f67f9d85-9439-4146-bfe0-079986f720ae',
 '0bdf7847-48c1-4fa0-81e8-666bb8f92943',
 'eeb8fab0-c0bf-4268-931e-882f7fd4a47d',
 '4bb1762c-ebc1-4625-ba84-611b4faeec75',
 'b74fd61b-e440-4f8b-91bc-cb4d94844f63',
 '7138e680-aad7-4dd5-9e98-c031175eaffa',
 '6ca21a3d-b6eb-4aac-9e0c-73bcb5a41a6b',
 '0c2b6bb0-8e78-4462-8937-1b62c6dae3bc',
 '08776c0d-8ad5-4cae-8e2d-028d4a9c4ad7',
 '53fd3b7d-0d30-49dc-ac2f-54ae5fea3c57',
 '608ce8fa-c020-41c7-b11b-09f1fa55af90',
 'cb61c79a-afcd-4d4c-bdb3-877b54fa1ad2',
 '495e8df2-fbf4-4617-878b-84aeda41f725',
 '1c06367e-9a14-461b-a3cc-1792387bb53c',
 'b06ff924-83dc-4309-a96a-fa5ed9bb3603',
 'ddf3816a-cf46-4d33-80d7-4151ce9967db',
 'bf5e8327-0b86-4bd1-895b-afc5fe65bdfd',
 'dec31508-2edf-4fa7-a645-2d814af50e35',
 '605c5a1d-24a8-4ce3-98ad-ed3bc78f5461',
 'e202ecfe-1ab7-

In [8]:
client.count_collections()

1

In [9]:
# client.delete_collection("ai_model_book")

#### Task 4: Access ChromaDB and perform vector search

**Objective:**

Use query to perform vector search against ChromaDB vector database

**Task Description:**

- define query
- run vector search
- print k=3 most similar documents

**Useful links:**

- [Langchain Query ChromaDB](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#query-directly)


In [10]:
search_query = "Types of Machine Learning Systems"

results = vector_db_from_client.similarity_search("search_query", k = 3,)

for res in results:
    print(res.page_content)
    print(res.metadata)
    print("\n")

Frame the Problem                                                                                                    39
Select a Performance Measure                                                                                  42
Check the Assumptions                                                                                             45
Get the Data                                                                                                                    45
Create the Workspace                                                                                                45
Download the Data                                                                                                    49
Take a Quick Look at the Data Structure                                                                50
Create a Test Set                                                                                                          54
Discover and Visualize the Data to Gain Insights